[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googlecolab/colab-samples/blob/main/notebooks/basic_notebook_features/text_cells.ipynb)




In [ ]:
# Install PyCaret from GitHub master branch
!pip install git+https://github.com/pycaret/pycaret.git@master --upgrade -q

In [ ]:
# Load Mall Customer Segmentation dataset
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Load dataset from URL
url = 'https://raw.githubusercontent.com/SteffiPeTaffy/machineLearningAZ/master/Machine%20Learning%20A-Z%20Template%20Folder/Part%204%20-%20Clustering/Section%2025%20-%20Hierarchical%20Clustering/Mall_Customers.csv'
df = pd.read_csv(url)

# Check columns first
print(f"Columns in dataset: {df.columns.tolist()}")

# Drop CustomerID if it exists
if 'CustomerID' in df.columns:
    df = df.drop(columns=['CustomerID'])

# Keep only numeric columns for clustering
df = df.select_dtypes(include=['number'])

print(f"\n✅ Mall Customers Dataset loaded: {df.shape}")
print(f"Features for clustering: {list(df.columns)}")
print(f"\nDataset Preview:")
df.head()

In [ ]:
# Explore the data before clustering
print("📊 Dataset Statistics:")
print(df.describe())

print(f"\n💡 We have {len(df)} customers with 3 features:")
print("   • Age: Customer age")
print("   • Annual Income (k$): Yearly income in thousands")
print("   • Spending Score (1-100): Shopping behavior score")

In [ ]:
from pycaret.clustering import *

# Initialize clustering environment
clust_exp = setup(
    data=df,
    session_id=789,
    normalize=True,
    transformation=True,
    pca=True,
    pca_components=2,
    verbose=False
)

In [ ]:
# Display all available clustering algorithms
print("🔍 Available Clustering Models:")
models()

In [ ]:
# Create K-Means clustering model with 5 clusters
print("🤖 Creating K-Means model with 5 customer segments...")
kmeans_model = create_model('kmeans', num_clusters=5)
print("✅ K-Means model created!")

In [ ]:
# Check if GPU is available
!nvidia-smi --query-gpu=name --format=csv,noheader

In [ ]:
# Assign cluster labels to customers
clustered_data = assign_model(kmeans_model)

print(f"✅ Customers segmented into {len(clustered_data['Cluster'].unique())} clusters")
print(f"\nCluster Distribution:")
print(clustered_data['Cluster'].value_counts().sort_index())

print("\n📊 Sample of clustered customers:")
clustered_data.head(10)

In [ ]:
# Generate clustering visualizations
print("📈 Generating cluster visualizations...")

plot_model(kmeans_model, plot='elbow')
plot_model(kmeans_model, plot='silhouette')
plot_model(kmeans_model, plot='cluster')
plot_model(kmeans_model, plot='distribution')

In [ ]:
# Analyze each cluster's characteristics
print("🔍 Cluster Characteristics:")
cluster_summary = clustered_data.groupby('Cluster').mean()
print(cluster_summary)

print("\n💡 Cluster Insights:")
for cluster in sorted(clustered_data['Cluster'].unique()):
    cluster_data = clustered_data[clustered_data['Cluster'] == cluster]
    avg_age = cluster_data['Age'].mean()
    avg_income = cluster_data['Annual Income (k$)'].mean()
    avg_spending = cluster_data['Spending Score (1-100)'].mean()
    print(f"\n   Cluster {cluster}: {len(cluster_data)} customers")
    print(f"      Avg Age: {avg_age:.1f} years")
    print(f"      Avg Income: ${avg_income:.1f}k")
    print(f"      Avg Spending Score: {avg_spending:.1f}/100")

In [ ]:
# Interactive dashboard for cluster evaluation
evaluate_model(kmeans_model)

In [ ]:
# Try another clustering algorithm - DBSCAN
print("🧩 Creating DBSCAN model...")
dbscan_model = create_model('dbscan')

dbscan_clustered = assign_model(dbscan_model)
print(f"\nDBSCAN found {len(dbscan_clustered['Cluster'].unique())} clusters")
plot_model(dbscan_model, plot='cluster')

In [ ]:
# Save the K-Means model
model_name = 'customer_segmentation_kmeans'
save_model(kmeans_model, model_name)

print(f"✅ Model successfully saved as '{model_name}.pkl'")
print(f"📦 Model can be loaded using: loaded = load_model('{model_name}')")

print(f"\n🎯 Clustering Summary:")
print(f"   • Dataset: Mall Customers (200 samples)")
print(f"   • Algorithm: K-Means")
print(f"   • Number of Clusters: 5")
print(f"   • Use Case: Customer segmentation for marketing")